<a target="_blank" href="https://colab.research.google.com/github/kurchi1205/ERA-S12-Assignment/blob/main/training.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [4]:
!pip install pytorch-lightning

     |████████████████████████████████| 722 kB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 163 kB 100.5 MB/s eta 0:00:01
     |████████████████████████████████| 729 kB 78.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 83.5 MB/s eta 0:00:01
     |████████████████████████████████| 228 kB 85.8 MB/s eta 0:00:01
     |████████████████████████████████| 114 kB 95.8 MB/s eta 0:00:01
     |████████████████████████████████| 269 kB 98.8 MB/s eta 0:00:01


In [ ]:
%pip install albumentations

In [1]:
!pip uninstall -y opencv-python
!conda install glib=2.51.0 -y
!pip install opencv-python
!pip install opencv-python-headless

Found existing installation: opencv-python 4.8.0.74
Uninstalling opencv-python-4.8.0.74:
  Successfully uninstalled opencv-python-4.8.0.74
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 23.7.2

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/studio-lab-user/.conda/envs/default

  added / updated specs:
    - glib=2.51.0


The following NEW packages will be INSTALLED:

  gettext            conda-forge/linux-64::gettext-0.21.1-h27087fc_0
  glib               conda-forge/linux-64::glib-2.51.0-2
  libiconv           conda-forge/linux-64::libiconv-1.17-h166bdaf_0
  pcre               conda-forge/linux-64::pcre-8.45-h9c3ff4c_0
  zlib               conda-forge/linux-64::zlib-1.2.13-h166bdaf_4

The following packages will be UPDATED:

  ca-certificates                

In [1]:
from data import CIFAR10LitModule
from train_lightning import LitResnet
import os
import torch
from pytorch_lightning import LightningModule, Trainer


In [2]:
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
AVAIL_GPUS

0

In [3]:
# init Dataloader from MNIST Dataloader
ds = CIFAR10LitModule(batch_size=32)
ds.setup()
train_loader = ds.train_dataloader()
resnet_model = LitResnet(0.01)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# initialize the trainer
trainer = Trainer(
    accelerator="auto",
    max_epochs = 20,
)

# Train the model
trainer.fit(resnet_model, train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
Loading `train_da

Training: 0it [00:00, ?it/s]